In [4]:
%cd ..

/home/alta/Conversational/OET/al826/2022/shortcuts/data_pruning


In [7]:
from src.helpers import DataLoader, DirHelper, Batcher

data_loader = DataLoader('bert', formatting=None)

train, dev, test = data_loader('twitter', 100)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1840/1840 [00:00<00:00, 8404.59it/s]
